In [21]:
import pandas as pd
import numpy as np
import sklearn

In [22]:
data_food = pd.read_csv("DataFood.csv")
data_country = pd.read_csv("DataCountry.csv")

In [23]:
# Before Processing
data_food.head()

,m49_code,country,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,reference,url,notes
0,104,Myanmar,NaN,23161.02,"Rice, milled",2015,1.78,1.78%,26.12kgs,Storage,Storage,"30 days storage, with trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
1,104,Myanmar,NaN,23161.02,"Rice, milled",2015,11.77,11.77%,88.18kgs,Storage,Storage,"60 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
2,104,Myanmar,NaN,23161.02,"Rice, milled",2015,5.88,5.88%,44.09kgs,Storage,Storage,"30 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
3,104,Myanmar,NaN,23161.02,"Rice, milled",2015,3.57,3.57%,52.24kgs,Storage,Storage,"60 days storage, with trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically
4,104,Myanmar,NaN,23161.02,"Rice, milled",2015,17.65,17.65%,132.27kgs,Storage,Storage,"90 days storage, no trapping",Rodents,NaN,Controlled Experiment,"Dr Steven Belmain (2015), context post-harvest...",NaN,Reference has been generated automatically


In [24]:
data_food = data_food.drop(columns=['reference', 'url', 'notes','country'])

In [25]:
data_food.dtypes

m49_code                      int64
region                       object
cpc_code                     object
commodity                    object
year                          int64
loss_percentage             float64
loss_percentage_original     object
loss_quantity                object
activity                     object
food_supply_stage            object
treatment                    object
cause_of_loss                object
sample_size                  object
method_data_collection       object
dtype: object

In [26]:
data_food.shape

(25416, 14)

In [27]:
# Before Processing
data_country.head()

,Country,combined figures (kg/capita/year),Household estimate (kg/capita/year),Household estimate (tonnes/year),Retail estimate (kg/capita/year),Retail estimate (tonnes/year),Food service estimate (kg/capita/year),Food service estimate (tonnes/year),Confidence in estimate,M49 code,Region,Source
0,Afghanistan,126,82,3109153,16,594982,28,1051783,Very Low Confidence,4,Southern Asia,https://www.unep.org/resources/report/unep-foo...
1,Albania,127,83,238492,16,45058,28,79651,Very Low Confidence,8,Southern Europe,https://www.unep.org/resources/report/unep-foo...
2,Algeria,135,91,3918529,16,673360,28,1190335,Very Low Confidence,12,Northern Africa,https://www.unep.org/resources/report/unep-foo...
3,Andorra,123,84,6497,13,988,26,1971,Low Confidence,20,Southern Europe,https://www.unep.org/resources/report/unep-foo...
4,Angola,144,100,3169523,16,497755,28,879908,Very Low Confidence,24,Sub-Saharan Africa,https://www.unep.org/resources/report/unep-foo...


In [28]:
data_country = data_country.drop(columns=['Source', 'Household estimate (kg/capita/year)','Retail estimate (kg/capita/year)','Food service estimate (kg/capita/year)'])
data_country['combined figures (tonnes/year)'] = data_country['Household estimate (tonnes/year)'] + data_country['Retail estimate (tonnes/year)'] + data_country['Food service estimate (tonnes/year)'] 
data_country = data_country.drop(columns=['Household estimate (tonnes/year)', 'Retail estimate (tonnes/year)', 'Food service estimate (tonnes/year)'])
data_country = data_country.rename(columns={"M49 code":"m49_code"})

In [29]:
data_country.dtypes

Country                              object
combined figures (kg/capita/year)     int64
Confidence in estimate               object
m49_code                              int64
Region                               object
combined figures (tonnes/year)        int64
dtype: object

In [35]:
# Merged data
data_food_country = pd.merge(data_food,data_country, on='m49_code')
data_food_country = data_food_country.drop(columns=['m49_code'])

In [36]:
data_food_country.head()

,region,cpc_code,commodity,year,loss_percentage,loss_percentage_original,loss_quantity,activity,food_supply_stage,treatment,cause_of_loss,sample_size,method_data_collection,Country,combined figures (kg/capita/year),Confidence in estimate,Region,combined figures (tonnes/year)
0,NaN,23161.02,"Rice, milled",2015,1.78,1.78%,26.12kgs,Storage,Storage,"30 days storage, with trapping",Rodents,NaN,Controlled Experiment,Myanmar,130,Very Low Confidence,South-eastern Asia,7005658
1,NaN,23161.02,"Rice, milled",2015,11.77,11.77%,88.18kgs,Storage,Storage,"60 days storage, no trapping",Rodents,NaN,Controlled Experiment,Myanmar,130,Very Low Confidence,South-eastern Asia,7005658
2,NaN,23161.02,"Rice, milled",2015,5.88,5.88%,44.09kgs,Storage,Storage,"30 days storage, no trapping",Rodents,NaN,Controlled Experiment,Myanmar,130,Very Low Confidence,South-eastern Asia,7005658
3,NaN,23161.02,"Rice, milled",2015,3.57,3.57%,52.24kgs,Storage,Storage,"60 days storage, with trapping",Rodents,NaN,Controlled Experiment,Myanmar,130,Very Low Confidence,South-eastern Asia,7005658
4,NaN,23161.02,"Rice, milled",2015,17.65,17.65%,132.27kgs,Storage,Storage,"90 days storage, no trapping",Rodents,NaN,Controlled Experiment,Myanmar,130,Very Low Confidence,South-eastern Asia,7005658


In [ ]:
data_food_country.food_supply_stage.unique()

array(['Storage', 'Harvest', 'Farm', nan, 'Post-harvest', 'Retail',
       'Wholesale', 'Trader', 'Whole supply chain', 'Processing',
       'Transport', 'Distribution', 'Households', 'Food Services',
       'Export', 'Pre-harvest', 'Market', 'Collector', 'Stacking',
       'Packing', 'Grading'], dtype=object)

In [ ]:
# Some data were lost because rows do not match
data_food_country.shape

(25347, 19)

In [ ]:
from sklearn.datasets import load_iris
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.pipeline import Pipeline


In [37]:
categorical_features = ['commodity', 'region', 'activity', 'treatment', 'cause of loss','country' ]


In [ ]:
target = 'food_supply_stage'
features = data_food_country.drop(columns=[target])
X_train, X_test, y_train, y_test = train_test_split(features, data_food_country[target], test_size=0.2, random_state=42)
model = LogisticRegression()
model.fit(X_train,y_train)
predictions = model.predict(X_test)
accuracy = accuracy_score(y_test, predictions)
print("Acc: ")

ValueError: could not convert string to float: 'Birgunj, Nepal to Patna'